# Applied Data Science Capstone - Cousera
---

## _(this notebook will be used specifically for capstone project)_
***

In [214]:
import pandas as pd
import numpy as np

In [215]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Segmenting and Clustering Neighborhoods in Toronto - _Part - 2_
---

In [7]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [1]:
!conda install -c conda-forge lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m          conda-forge

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0                       --

### Create BeautifulSoup object for Scraping Webpage

In [217]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)

soup=BeautifulSoup(r.text,'lxml')

### Scrape the table from Wiki

In [218]:
match=soup.find('table',class_='wikitable sortable')

A=[]
B=[]
C=[]
rows = match.find_all('tr')
for row in rows:
    cols=row.find_all('td')
    for i,x in enumerate(cols):
            if i==0:
                A.append(x.text.strip())
            elif i==1:
                B.append(x.text.strip())
            else:
                C.append(x.text.strip())
    

### Creating a Dataframe from the scraped data

In [221]:
#Create a distionary
Dict={'Postal':A,'Borough':B,'Neighborhood':C}

#create a panda Dataframe
DF=pd.DataFrame(Dict)
#remove borough with value 'Not assigned'
DF=DF[DF['Borough']!='Not assigned'].reset_index(drop=True)
#replace '/' with ',' in the neighborhood string
DF['Neighborhood']=DF['Neighborhood'].replace(" / ",",",regex=True)

In [222]:
DF.shape

(103, 3)